In [1]:
import os
print(os.getcwd())


D:\Python\py_programs\TaskPrioritization_NLP\models\bert


In [1]:
# Ячейка 1: Импорт библиотек
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [2]:
# Ячейка 2: Загрузка данных
df = pd.read_csv("processed_data.csv", sep=';', encoding='utf-8')  
texts = df['Обработанный_Текст']  # Столбец с текстами
labels = df['Метка']  # Столбец с метками

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)
print(type(train_texts))  # Должно быть <class 'list'>
print(type(train_texts[0]))  # Должно быть <class 'str'>

<class 'pandas.core.series.Series'>
<class 'str'>


In [3]:
# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Функция для токенизации
def tokenize_data(texts):
    # Убедимся, что данные являются списком строк
    texts = [str(text) for text in texts if text is not None]
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Преобразуем Series в список строк и удаляем None
train_texts_list = [str(text) for text in train_texts.tolist() if text is not None]
val_texts_list = [str(text) for text in val_texts.tolist() if text is not None]
print(type(train_texts_list))  # Должно быть <class 'list'>
print(type(val_texts_list))

# Токенизация данных
train_encodings = tokenize_data(train_texts_list)
val_encodings = tokenize_data(val_texts_list)


<class 'list'>
<class 'list'>


In [4]:
# Ячейка 4: Создание датасетов
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextDataset(train_encodings, train_labels.tolist())
val_dataset = TextDataset(val_encodings, val_labels.tolist())

In [5]:
# Ячейка 5: Создание модели и обучение
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Бинарная классификация

training_args = TrainingArguments(
    output_dir='training/results',           # Куда сохранять результаты
    num_train_epochs=3,               # Количество эпох
    per_device_train_batch_size=16,   # Размер батча
    per_device_eval_batch_size=64,    # Размер батча для валидации
    warmup_steps=500,                 # Линейный LR scheduler
    weight_decay=0.01,                # Регуляризация
    logging_dir='training/logs',             # Логи
    logging_steps=10,
    evaluation_strategy="epoch",      # Оценка после каждой эпохи
    save_strategy="epoch"             # Сохранение модели после каждой эпохи
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids, p.predictions.argmax(-1))}
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
D:\Python\py_programs\TaskPrioritization_NLP\venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\sha-b\AppData\Local\Temp\ipykernel_2580\1512811855.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy
1,0.444500,0.384038,0.822797
2,0.332900,0.254734,0.895594
3,0.230700,0.255317,0.902299


C:\Users\sha-b\AppData\Local\Temp\ipykernel_2580\1512811855.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\sha-b\AppData\Local\Temp\ipykernel_2580\1512811855.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\sha-b\AppData\Local\Temp\ipykernel_2580\1512811855.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in s

TrainOutput(global_step=783, training_loss=0.36306491924276146, metrics={'train_runtime': 4629.3319, 'train_samples_per_second': 2.704, 'train_steps_per_second': 0.169, 'total_flos': 823471825512960.0, 'train_loss': 0.36306491924276146, 'epoch': 3.0})

In [ ]:
# Ячейка 6: Оценка модели и сохранение
trainer.evaluate()

model.save_pretrained("training/saved_model")
tokenizer.save_pretrained("training/saved_model")

C:\Users\sha-b\AppData\Local\Temp\ipykernel_2580\1512811855.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
